In [1]:
%load_ext autoreload
%autoreload 2
from sound_field import SoundField,SoundFieldDataset,divide_to_subbands,divide_to_time_windows
from signal_info import signal_info
from optimizer import optimizer
from optimizer_v2 import optimizer_v2
from DoA_est import DoA_via_bands
import utils
import numpy as np
import math
import time
import torch
import gc
import os
from collections import defaultdict,Counter
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import time
import concurrent.futures



LEBEDEV = 'lebedev'
POINTS_162 = '162_points'
SQP_OPT_METHOD = "SQP"
GD_DEEP_OPT_METHOD = "GD_Deep"
GD_OPT_METHOD = "GD_lagrange_multi" #GD_Deep GD_lagrange_multi
SLS_OPT_METHOD = "SLS"
if torch.cuda.is_available():
    DEVICE = torch.device("cuda:0")
    print("Device Name:", torch.cuda.get_device_name(DEVICE))
else:
    DEVICE = torch.device("cpu")
    print("Device Name: CPU")
torch.set_grad_enabled(False)
torch.cuda.empty_cache()  # Free up unused memory on GPU
gc.collect()  # Clean up unused memory on CPU


Device Name: NVIDIA GeForce RTX 4080 SUPER


56

In [2]:
data_set = torch.load(r"data\WSJ0\Dataset_03_01_25.pt",weights_only=False)

In [3]:
train_set = SoundFieldDataset(data_set['train'],DEVICE)
val_set = SoundFieldDataset(data_set['validation'],DEVICE)
test_set = SoundFieldDataset(data_set['test'],DEVICE)

In [4]:
train_set.__dict__.keys()

dict_keys(['data', 'device', 'input_order', 'upscaled_order', 'num_bins', 'window_length', 'max_num_windows', 'min_num_speakers', 'max_num_speakers', 'min_theta', 'max_theta', 'min_phi', 'max_phi', 'min_dist', 'sr', 'grid_type', 'P_th', 'P_ph'])

In [5]:
batch_size = 1
num_epochs = 100

down_sample = 1
num_bins = 45

window_length = 1024
max_num_windows = int(1e10)

mask = None

train_data_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_set, batch_size=len(val_set), shuffle=True)
test_data_loader = DataLoader(test_set, batch_size=len(test_set), shuffle=True)

Y_p = utils.create_sh_matrix(train_set.input_order, zen=train_set.P_th, azi=train_set.P_ph).to(DEVICE)
Y_p_tag = utils.create_sh_matrix(3, zen=train_set.P_th, azi=train_set.P_ph).to(DEVICE) # upscaling matrix (TODO ADD UPSCALE ORDER TO DATASET INFO)
opt_model = optimizer_v2(Y_p=Y_p,alpha=0.01,num_iters=5,device=DEVICE)


In [6]:
def preprocess_batch(batch, num_bins, down_sample, window_length, max_num_windows,sr):
    a_nmt_subbands = divide_to_subbands(batch[0], num_bins=num_bins, downsample=down_sample, sr=sr)
    a_nmt_subbands_windowed = divide_to_time_windows(a_nmt_subbands,window_length=window_length, max_num_windows=max_num_windows)

    gt_a_nmt_subbands = divide_to_subbands(batch[1], num_bins=num_bins, downsample=down_sample,sr=sr)
    gt_a_nmt_subbands_windowed = divide_to_time_windows(anm_t_subbands=gt_a_nmt_subbands,window_length=window_length, max_num_windows=max_num_windows)
    print("Finished Preprocessing")
    return a_nmt_subbands_windowed,gt_a_nmt_subbands_windowed

In [7]:
if False:
    for epoch in range(num_epochs):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            next_preprocessed_batch = None
            data_loader_iter = iter(train_data_loader)
            curr_batch_idx = 0
            while curr_batch_idx < len(train_data_loader):
                # Preprocess the current batch on the CPU
                if next_preprocessed_batch is None:
                    start = time.time()
                    low_order_subbands_windowed,high_order_subbands_windowed = preprocess_batch(next(data_loader_iter), num_bins, down_sample, window_length, max_num_windows,train_set.sr)
                    print("Preprocessing Time:",time.time()-start)
                else:
                    low_order_subbands_windowed,high_order_subbands_windowed = next_preprocessed_batch.result()

                # Start preprocessing the next batch on the CPU
                if curr_batch_idx < len(train_data_loader) - 1:
                    next_batch = next(data_loader_iter)
                    next_preprocessed_batch = executor.submit(preprocess_batch, next_batch, num_bins, down_sample, window_length, max_num_windows)
                
                print("Starting on GPU")
                est_high_order_subbands_windowed = opt_model.forward(low_order_subbands_windowed,mask)

                curr_batch_idx += 1


                # Do something with the preprocessed batch
                break

In [9]:
for epoch in range(num_epochs):
    for batch in train_data_loader:
        low_order_anmt,high_order_anmt = batch
        a_nmt_subbands = divide_to_subbands(low_order_anmt, num_bins=num_bins, downsample=down_sample, sr=train_set.sr)
        a_nmt_subbands_windowed = divide_to_time_windows(a_nmt_subbands,window_length=window_length, max_num_windows=max_num_windows)
        gt_a_nmt_subbands = divide_to_subbands(high_order_anmt, num_bins=num_bins, downsample=down_sample,sr=train_set.sr)
        gt_a_nmt_subbands_windowed = divide_to_time_windows(anm_t_subbands=gt_a_nmt_subbands,window_length=window_length, max_num_windows=max_num_windows)
        est_sparse_dict_subbands = opt_model.forward(a_nmt_subbands_windowed,mask).permute(0,1,3,2) #change to (#Windows,#Bands,WindowLength,P)
        est_upscaled_subbands_windowed = torch.matmul(est_sparse_dict_subbands,Y_p_tag.t())
        a=1




c:\Users\amitmils\Documents\Repo\AmbisonicUpscaling\sound_field.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  anm_t_subbands[:, :, coeff] = torch.tensor(erb_bank.subbands.T).clone()
100%|██████████| 5/5 [00:00<?, ?it/s]


RuntimeError: self must be a matrix